In [11]:
import numpy
import math
from matplotlib import pyplot

''' Atomic fractional coordiantes x/a, y/b, and z/c
from (1988 Macromolecules Immiczi and Iannelil) 
Comments indicate bonding. '''
C1 = numpy.array([[-0.0727, 0.2291, 0.2004],  # 0: 1
                  [-0.0765, 0.1592, 0.2788],  # 1: 0 2
                  [-0.1021, 0.1602, 0.5098],  # 2: 1 4
                  [-0.3087, 0.0589, 0.4941],  # 3: 4
                  [-0.1146, 0.0928, 0.6057],  # 4: 2 3 5
                  [-0.1044, 0.0854, 0.8428],  # 5: 4 7
                  [ 0.2775, 0.0797, 0.9260],  # 6: 7
                  [ 0.0872, 0.1156, 0.9730],  # 7: 5 6 8
                  [ 0.1026, 0.1221, 1.2109]]) # 8: 7
''' Angle between c vector and x-axis. '''
beta = 99.5 * numpy.pi / 180.0
''' Columns of unit_cell are the a, b, and c cell vectors. '''
unit_cell = numpy.array([[6.63,  0.00, 6.50*numpy.cos(beta)],
                         [0.00, 20.78, 0.0],
                         [0.00,  0.00, 6.50*numpy.sin(beta)]])


def apply_space_group(C1):
    ''' Edited symmetry operations to make figure match - not checked '''
    C2, C3, C4 = C1.copy(), C1.copy(), C1.copy()

    C2[:,0] = -C2[:,0]
    C2[:,1] = -C2[:,1]
    C2[:,2] = -C2[:,2] + 1.0

    C3[:,0] =  C3[:,0] - 0.5
    C3[:,1] =  C3[:,1] - 0.5
    C3[:,2] =  C3[:,2]

    C4[:,0] =  C4[:,0] - 0.5
    C4[:,1] = -C4[:,1] + 0.5
    C4[:,2] = -C4[:,2] + 1.0
    return C1, C2, C3, C4  

def apply_hydrogens(C):
    ''' Construction of hydrogens from Theodorou (MD of aPP Melts) with 
    l_H=0.11 nm and theta_H=1.28 rad and c=+1 in C1 & C4 and c=-1 in C2 & C3'''   
    
    H = numpy.empty([9,3])       

    l_H = 0.11
    theta_H = 1.28
    c=1

    # Adding hydrogens 
    # C1    
    def chiral_H_direction(i,j,k):      
        return unit_vector(numpy.cross(C[i,:]-C[j,:], C[i,:]-C1[k,:]))
              
    # Methyl Group Hydrogens    
    '''
    for i in range(7):
      for j in range(9,12):
        if j==8:
          H[i+j,0]=C[i,0]+1.037
          H[i+j,1]=C[i,1]+0          

        if j==9:
          H[i+j,0]=C[i,0]-0.5185
          H[i+j,1]=C[i,1]+0.898

        if j==10:
          H[i+j,0]=C[i,0]-0.5185
          H[i+j,1]=C[i,1]-0.898
          
        H[i+j,2]=C[i,2]-0.37
      i=i+3
    '''

    #Non-Methyl Hydrogens
    b = [None]*3
    b[0] = unit_vector(C[2,:] - C[1,:])
    b[1] = unit_vector(C[5,:] - C[4,:])
    b[2] = unit_vector(C[8,:] - C[7,:])

    v3 = numpy.cross(b[0],b[2])/numpy.linalg.norm(numpy.cross(b[0],b[2]))
    u3 = (b[0] - b[2]) / math.sqrt(2.0*(1.0 - b[0].dot(b[2])))

    H[0,:] = C[1,:] + c*l_H*chiral_H_direction(0,2,8)
    H[1,:] = C[7,:] + c*l_H*chiral_H_direction(6,8,5)
    H[2,:] = C[4,:] + c*l_H*chiral_H_direction(3,5,2)   

    H[3,:] = C[2,:]+l_H*(numpy.sin(theta_H/2)*u3+numpy.cos(theta_H/2)*v3)
    H[4,:] = C[2,:]+l_H*(numpy.sin(theta_H/2)*u3-numpy.cos(theta_H/2)*v3)
    H[5,:] = C[8,:]+l_H*(numpy.sin(theta_H/2)*u3+numpy.cos(theta_H/2)*v3)
    H[6,:] = C[8,:]+l_H*(numpy.sin(theta_H/2)*u3-numpy.cos(theta_H/2)*v3)
    H[7,:] = C[5,:]+l_H*(numpy.sin(theta_H/2)*u3+numpy.cos(theta_H/2)*v3)
    H[8,:] = C[5,:]+l_H*(numpy.sin(theta_H/2)*u3-numpy.cos(theta_H/2)*v3)
    
    return H

def perfect_crystal(a, b, c):
    ''' Creating the initial crystalline system - a & b & c are integers ''' 
    
  
def plot_unit_cell():
    C11,C2,C3,C4 = apply_space_group(C1)

    x = numpy.vstack([numpy.dot(x, unit_cell) for x in apply_space_group(C1)])
    lx, ly = unit_cell[0,0], unit_cell[1,1]

    figure = pyplot.figure(figsize=(9,5))
    ax = pyplot.subplot(221)
    ax.plot(x[:9,1],    -x[:9,0],    '.', ms=30, alpha=0.5, label='C1')
    ax.plot(x[9:18,1],  -x[9:18,0],  '.', ms=30, alpha=0.5, label='C2')
    ax.plot(x[18:27,1], -x[18:27,0], '.', ms=30, alpha=0.5, label='C3')
    ax.plot(x[27:36,1], -x[27:36,0], '.', ms=30, alpha=0.5, label='C4')
    ax.set_xlabel('y')
    ax.set_ylabel('-x')
    ax.set_aspect('equal')
    pyplot.legend(ncol=4, bbox_to_anchor=(0,1), loc='lower left')

    ax = pyplot.subplot(222)
    ax.plot(x[:9,2],    -x[:9,0],    '.', ms=30, alpha=0.5, label='C1')
    ax.plot(x[9:18,2],  -x[9:18,0],  '.', ms=30, alpha=0.5, label='C2')
    ax.plot(x[18:27,2], -x[18:27,0], '.', ms=30, alpha=0.5, label='C3')
    ax.plot(x[27:36,2], -x[27:36,0], '.', ms=30, alpha=0.5, label='C4')
    ax.set_xlabel('z')
    ax.set_ylabel('-x')
    ax.set_aspect('equal')

    ax = pyplot.subplot(223)
    ax.plot(x[:9,1],    x[:9,2],    '.', ms=30, alpha=0.5, label='C1')
    ax.plot(x[9:18,1],  x[9:18,2],  '.', ms=30, alpha=0.5, label='C2')
    ax.plot(x[18:27,1], x[18:27,2], '.', ms=30, alpha=0.5, label='C3')
    ax.plot(x[27:36,1], x[27:36,2], '.', ms=30, alpha=0.5, label='C4')
    ax.set_xlabel('y')
    ax.set_ylabel('z')
    ax.set_aspect('equal')
    pyplot.tight_layout()

    H = numpy.concatenate((apply_hydrogens(C1), apply_hydrogens(C2), apply_hydrogens(C3), apply_hydrogens(C4)),axis=0)    
    figure1 = pyplot.figure(figsize=(9,5))
    aH = pyplot.subplot(111)
    aH.plot(H[:9,1],      -H[:9,0],      '.', ms=30, alpha=0.5, label='H1')
    aH.plot(H[9:18,1],    -H[9:18,0],    '.', ms=30, alpha=0.5, label='H2')
    aH.plot(H[18:27,1],   -H[18:27,0],   '.', ms=30, alpha=0.5, label='H3')
    aH.plot(H[27:36,1],    -H[27:36,0],   '.', ms=30, alpha=0.5, label='H4')
    aH.set_xlabel('y')
    aH.set_ylabel('-x')
    aH.set_aspect('equal')
    pyplot.legend(ncol=4, bbox_to_anchor=(0,1), loc='lower left')

    
def unit_vector(v):
    return v / numpy.linalg.norm(v)



In [12]:
# Determine which atoms are bonded to each other.
r = numpy.dot(C1, unit_cell)
for i in range(len((r))):
    s = '{}:'.format(i)
    for j in range(len(r)):
        d = numpy.linalg.norm(r[i,:]-r[j,:])
        if i != j and d < 1.6:
            s += ' {}'.format(j)
    print(s)
    

0: 1
1: 0 2
2: 1 4
3: 4
4: 2 3 5
5: 4 7
6: 7
7: 5 6 8
8: 7
